In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        if "train.csv" in filename:
            df_train = pd.read_csv(os.path.join(dirname, filename))
        elif "test.csv" in filename:
            df_test = pd.read_csv(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train.head()

In [ ]:
df_train_analyze = df_train.copy(deep=True)
df_train_analyze["text_len"] = df_train_analyze["excerpt"].str.len()
df_train_analyze.head()

In [ ]:
df_train_analyze.describe()

# Word frequency analysis

## Initial thought: the word frequency of certain words may influence the final score

In [ ]:
import nltk
from nltk.tokenize import TreebankWordTokenizer

df_train_token = df_train_analyze.copy(deep=True)
for i, row in df_train_analyze.iterrows():
    excerpt = row["excerpt"].lower()
    punc_tokenizer = nltk.RegexpTokenizer(r"\w+")
    alpha_tokenizer = nltk.RegexpTokenizer(r"[a-zA-Z\-]+")
    word_tokenizer = TreebankWordTokenizer()
    new_words = punc_tokenizer.tokenize(excerpt)
    new_words = " ".join(new_words)
    new_words = alpha_tokenizer.tokenize(new_words)
    new_words = " ".join(new_words)
    new_words = word_tokenizer.tokenize(new_words)
    df_train_token.loc[i, "token"] = " ".join(new_words) 

In [ ]:
df_train_token.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

ex_list = df_train_token["token"].values.tolist()
vectorizer = CountVectorizer()
word_vector = vectorizer.fit_transform(ex_list)
# print(vectorizer.get_feature_names())
word_freq_array = word_vector.toarray()
print(word_freq_array)

In [ ]:
import statsmodels.api as sm

endog = np.array(df_train_token["target"].values.tolist())
mod = sm.OLS(endog, word_freq_array)
res = mod.fit()
print(res.summary())